In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import time
import random
import numpy as np
import pandas as pd
import librosa

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.python.platform import gfile

In [0]:
PATH_TRAIN = "/content/gdrive/My Drive/train"
PATH_TEST = "/content/gdrive/My Drive/test"
BATCH_SIZE = 19*14
ITERATIONS = 800
ITERATIONS_TEST = 10
EVAL_EVERY = 5
NUM_LABELS = 0
LEARNING_RATE = 1E-4
LOGDIR = 'log/'
TEST_LOGDIR ='log_test/'
GSAVE = '/content/gdrive/My Drive/c/graph/'
LABEL_TO_INDEX_MAP={}
n_fft =1024
hop_length = 512
song_samples = 660000


In [0]:
def init(path):
  labels = os.listdir(path)
  index = 0
  for label in labels:
    LABEL_TO_INDEX_MAP[label]=index
    index +=1
    
  global NUM_LABELS
  NUM_LABELS = len(LABEL_TO_INDEX_MAP)

In [0]:
def one_hot_encoding(label):
  encoding =[0] * len(LABEL_TO_INDEX_MAP)
  encoding[LABEL_TO_INDEX_MAP[label]] = 1
  return encoding

In [0]:
#def splitsongs(X, y, window = 0.1, overlap = 0.5):
def splitsongs(X, window = 0.1, overlap = 0.5):
    # Empty lists to hold our results
    temp_X = []
    temp_y = []

    # Get the input song array size
    xshape = X.shape[0]
    chunk = int(xshape*window)
    offset = int(chunk*(1.-overlap))
    
    # Split the song and create new ones on windows
    spsong = [X[i:i+chunk] for i in range(0, xshape - chunk + offset, offset)]
    for s in spsong:
        temp_X.append(s)
        #temp_y.append(y)
    #return np.array(temp_X), np.array(temp_y)
    return np.array(temp_X)

In [0]:
def to_melspectrogram(songs, n_fft = 1024, hop_length = 512):
    
    # Transformation function
    melspec = lambda x: librosa.feature.melspectrogram(x, n_fft = n_fft,
        hop_length = hop_length)[:,:,np.newaxis]

    # map transformation of input songs to melspectrogram using log-scale
    tsongs = map(melspec, songs)
    return np.array(list(tsongs))

In [0]:
def main():
  X =[]
  Y =[]
  #random.seed(5896)
  path= "/content/gdrive/My Drive/test_demo/a/Hey_Sailor.mp3"
  
  #path = os.path.join(path,'*','*.mp3')
  
  waves = gfile.Glob(path)

  #while True:  
  #random.shuffle(waves)
  for wave_path in waves:
      #_,label = os.path.split(os.path.dirname(wave_path))
      signal, sr = librosa.load(wave_path)
      signal = signal[:song_samples]
  
    
      #signals, y = splitsongs(signal, one_hot_encoding(label))
      signals = splitsongs(signal)
     

      specs = to_melspectrogram(signals, n_fft, hop_length)
      print(wave_path)
      
      X.extend(specs)
      
  M=np.array(X)
  print(M.shape)
  M=np.reshape(X,(-1,128,129))
  print(M.shape)
  np.save('/content/gdrive/My Drive/test_demo/samples',np.array(M))
  #np.save('/content/gdrive/My Drive/test_demo/genres',np.array(Y))
    
   
        

In [9]:
if __name__=='__main__':
    init(PATH_TRAIN)
    main()

/content/gdrive/My Drive/test_demo/a/Hey_Sailor.mp3
(19, 128, 129, 1)
(19, 128, 129)
